## 모델 훈련

## 1. 모듈 불러오기

In [ ]:
import torch
from models.preprocess import train_loader, test_loader
from models.optimizers import (
    criterion, 
    shallow_optimizer, 
    deep5_optimizer, 
    deep10_optimizer,
    skipcondeep10_optimizer,
    resnet18_optimizer,
    shallownet,
    deepnet5,
    deepnet10,
    skipcondeep10,
    resnet18
)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

## 2. 모델 평가를 위한 변수 설정

In [ ]:
MODELNAME = [
    "ShallowNet", 
    "DeepNet5", 
    "DeepNet10", 
    "SkipConnectedDeep10",
    "ResNet18"
    ]

Models = [
    shallownet, 
    deepnet5, 
    deepnet10, 
    skipcondeep10,
    resnet18
    ]

Optimizer = [
    shallow_optimizer, 
    deep5_optimizer, 
    deep10_optimizer, 
    skipcondeep10_optimizer,
    resnet18_optimizer
    ]

Trainerror = [
    shallow_train_loss:=list(), 
    deep5_train_loss:=list(), 
    deep10_train_loss:=list(), 
    skipcondeep10_train_loss:=list(),
    resnet18_train_loss:=list()
    ]

Testerror = [
    shallow_test_loss:=list(), 
    deep5_test_loss:=list(), 
    deep10_test_loss:=list(), 
    skipcondeep10_test_loss:=list(),
    resnet18_test_loss:=list()
    ]

Accuracy = [
    shallow_accuracy:=list(), 
    deep5_accuracy:=list(), 
    deep10_accuracy:=list(), 
    skipcondeep10_accuracy:=list(),
    resnet18_accuracy:=list()
    ]

## 3. 모델 훈련

In [ ]:
def main():
    for idx in range(len(Models)):
        epochs = 30
        model = Models[idx].to(device)
        optimizer = Optimizer[idx]
        model_train_loss = Trainerror[idx]
        model_test_loss = Testerror[idx]
        model_accuracy = Accuracy[idx]

        print(f"\n{MODELNAME[idx]} Model Training\n-------------------------------------------")

        for epoch in range(epochs):  
            model.train()
            running_loss = 0.0
            for i, data in enumerate(train_loader, 0):
                inputs, labels = data
                inputs = inputs.to(device)
                labels = labels.to(device)
                optimizer.zero_grad()
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                loss.backward()
                optimizer.step()
                running_loss += loss.item()

            print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / (i+1):.3f}')

            model_train_loss.append(running_loss / (i+1))
            running_loss = 0.0

            model.eval()
            correct, total = 0, 0
            
            with torch.no_grad():
                for i, data in enumerate(test_loader, 0):
                    images, labels = data
                    inputs = inputs.to(device)
                    labels = labels.to(device)
                    outputs = model(images)
                    loss = criterion(outputs, labels)
                    _, predicted = torch.max(outputs.data, 1)

                    total += labels.size(0)
                    correct += (predicted == labels).sum().item()
                    running_loss += loss.item()

                model_test_loss.append(running_loss / (i+1))
                model_accuracy.append(100 * correct // total)

In [ ]:
main()

## 4. 모델 저장

In [ ]:
for i in range(len(Models)):
    PATH = f'./{MODELNAME[i]}.pth'
    torch.save(Models[i].state_dict(), PATH)

In [ ]:
# 이전에 학습했던 오차, 정확도를 불러옵니다.
li = []

with open("model_output.txt", "r") as file:
    for fi in file:
        ll = [float(name.strip()) for name in fi.split(",")]
        li.append(ll)

for i in range(4):
    Trainerror[i] += li[3*i]
    Testerror[i] += li[3*i+1]
    Accuracy[i] += li[3*i+2]

## 5. 그래프 작성

In [ ]:
import matplotlib.pyplot as plt
import plotly.graph_objects as go

epoch_list = [i+1 for i in range(len(shallow_train_loss))]
y_list = [Trainerror, Testerror, Accuracy]
title_list = ["Training Error", "Test Error", "Test Accuracy"]
ylabel_list = ["training error", "test Error", "accuracy"]

In [ ]:
for i in range(len(y_list)):
    y = y_list[i]
    title = title_list[i]
    ylabel = ylabel_list[i]

    plt.figure(i+1)
    #for j in range(len(Model)):
    for j in range(4):
        plt.plot(
            epoch_list, 
            y[j], 
            label=MODELNAME[j]
            )

        plt.title(title) 
        plt.xlabel("Epoch") 
        plt.ylabel(ylabel)
        plt.legend()

In [ ]:
fig = go.Figure()

for i in range(len(Models)):
    fig.add_trace(go.Scatter(
        x=epoch_list, 
        y=Trainerror[i],
        mode='lines',
        name=MODELNAME[i]
        ))

fig.update_layout(title_text="Training Error")
fig.update_xaxes(title_text='Epoch')
fig.update_yaxes(title_text='Training Error')

In [ ]:
fig = go.Figure()

for i in range(len(Models)):
    fig.add_trace(go.Scatter(
        x=epoch_list, 
        y=Testerror[i],
        mode='lines',
        name=MODELNAME[i]
        ))

fig.update_layout(title_text="Test Error")
fig.update_xaxes(title_text='epoch')
fig.update_yaxes(title_text='test error')

In [ ]:
fig = go.Figure()

for i in range(len(Models)):
    fig.add_trace(go.Scatter(
        x=epoch_list, 
        y=Accuracy[i],
        mode='lines',
        name=MODELNAME[i]
        ))

fig.update_layout(title_text="Accuracy")
fig.update_xaxes(title_text='epoch')
fig.update_yaxes(title_text='accuracy')

## 6. 모델 학습 결과 요약

In [ ]:
print("_______________________________ S U M M A R Y ________________________________")
for i in range(len(Models)):
    min_loss_index, min_loss = Testerror[i].index(min(Testerror[i])), min(Testerror[i])
    max_accuracy_index, max_accuracy = Accuracy[i].index(max(Accuracy[i])), max(Accuracy[i]) 

    print(f"\n{MODELNAME[i]}\n------------------------------------------------------------------------------")
    print(f"Min Error: (epoch {min_loss_index}, loss {min_loss:.3f})  |  Max Accuracy: (epoch {max_accuracy_index}, accuracy {max_accuracy} %)")

In [ ]:
from models.preprocess import classes
from models.models import (
    ShallowNet,
    DeepNet5,
    DeepNet10,
    SkipConDeep10
)

Modelclass = [
    ShallowNet(),
    DeepNet5(),
    DeepNet10(),
    SkipConDeep10()
]

In [ ]:
print("<Accuracy of Each Class>")

for i in range(len(Models)):
    print(f"\n{MODELNAME[i]}\n--------------------------")

    net = Modelclass[i]
    PATH = f'./{MODELNAME[i]}.pth'
    net.load_state_dict(torch.load(PATH))

    correct_pred = {classname: 0 for classname in classes}
    total_pred = {classname: 0 for classname in classes}
    with torch.no_grad():
        for data in test_loader:
            images, labels = data
            images = images.to(device)
            labels = labels.to(device)
            outputs = net(images)
            _, predictions = torch.max(outputs, 1)

            for label, prediction in zip(labels, predictions):
                if label == prediction:
                    correct_pred[classes[label]] += 1
                total_pred[classes[label]] += 1

    for classname, correct_count in correct_pred.items():
        accuracy = 100 * float(correct_count) / total_pred[classname]
        print(f'{classname:15s}: {accuracy:.1f} %')